In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: data.pickle             
  inflating: label.pickle            


In [0]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf

x = pickle.load(open('data.pickle','rb'))
y = pickle.load(open('label.pickle','rb'))

In [6]:
print(np.shape(x))
x=np.reshape(x,(501,299,299,3))
print(np.shape(x))

(501, 1, 299, 299, 3)
(501, 299, 299, 3)


In [7]:
print(np.shape(y))

(501,)


In [0]:
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import Recall,Precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Activation,Conv2D,MaxPooling2D,Dropout


model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(299,299,3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())

model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01)))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dropout(0.2))

model.add(Dense(3))
model.add(Activation('softmax'))


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
import keras
from tensorflow.keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint,TensorBoard
filename="metrics.csv"
es=EarlyStopping(monitor='loss', min_delta=0, patience=3, mode='min', baseline=None, restore_best_weights=True)
csv=CSVLogger(filename, separator=',', append=False)
checkpoint=ModelCheckpoint("weights.{epoch}-{val_acc:.2f}.hdf5", monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
tensorboard=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

In [17]:
model.fit(x, y, epochs=25, batch_size=64, callbacks=[tensorboard])

Train on 501 samples
Epoch 1/25
501/501 [==============================] - 11s 21ms/sample - loss: 1.9103 - acc: 0.8942
Epoch 2/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.8671 - acc: 0.8842
Epoch 3/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.7346 - acc: 0.9381
Epoch 4/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.6344 - acc: 0.9521
Epoch 5/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.5641 - acc: 0.9641
Epoch 6/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.4730 - acc: 0.9681
Epoch 7/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.4049 - acc: 0.9780
Epoch 8/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.3472 - acc: 0.9741
Epoch 9/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.2769 - acc: 0.9860
Epoch 10/25
501/501 [==============================] - 6s 13ms/sample - loss: 1.2195 - acc: 0.99

In [18]:
model.evaluate(x, y, verbose=1)

501/501 [==============================] - 3s 5ms/sample - loss: 0.7471 - acc: 1.0000


[0.7470957473842446, 1.0]

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 297, 297, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 146, 146, 128)     73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 128)       147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 156800)           

In [0]:
model.save('model_1.h5')

In [21]:
!pwd

/content


In [0]:
import zipfile
import os
def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [0]:
zipfolder('logs',"./logs")

In [0]:
!rm -rf logs